In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Load the dataset
data_chunks = pd.read_csv("car_prices.csv", chunksize=1000)  # Chunksize of 1000 rows

# Initialize lists to store accuracy scores and predictions
accuracy_scores = []
all_predictions = []

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['year']),  # Assuming 'year' is the only numerical feature
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'color', 'interior', 'seller', 'saledate'])
    ],
    remainder='passthrough'
)

# Initialize Random Forest model
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Perform 10-fold cross-validation on each chunk
for i, chunk in enumerate(data_chunks, 1):
    print(f"Processing chunk {i}...")
    # Drop rows with missing values
    chunk = chunk.dropna()

    # Extract features and target variable
    X = chunk.drop(columns=["sellingprice"])  # Features
    y = chunk["sellingprice"]  # Target variable

    # Initialize KFold
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Initialize lists to store accuracy scores
    accuracy_scores_chunk = []

    # Perform 10-fold cross-validation
    for train_index, test_index in kf.split(X):
        # Split data into train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the model
        rf_model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = rf_model.predict(X_test)

        # Calculate accuracy (R-squared score)
        accuracy = rf_model.score(X_test, y_test)

        # Append accuracy to list
        accuracy_scores_chunk.append(accuracy)

    # Calculate average accuracy for the chunk
    avg_accuracy_chunk = np.mean(accuracy_scores_chunk)
    print(f"Average accuracy for chunk {i}: {avg_accuracy_chunk}")

    # Append average accuracy to list
    accuracy_scores.append(avg_accuracy_chunk)

    # Predict on the entire chunk
    chunk_predictions = rf_model.predict(X)

    # Store predictions in the list
    all_predictions.append(chunk_predictions)

    # Include all columns in predictions DataFrame
    predictions_df = pd.DataFrame({'sellingprice_predicted': chunk_predictions})
    predictions_df = pd.concat([chunk.drop(columns=["sellingprice"]), predictions_df], axis=1)

    # Save predictions to CSV file
    predictions_df.to_csv(f"random_forest_predictions_for_chunk_{i}.csv", index=False)

# Calculate net accuracy across chunks
net_accuracy = np.mean(accuracy_scores)
print(f"Net accuracy across chunks: {net_accuracy}")

# Combine all predictions into a single DataFrame
all_predictions_df = pd.concat([pd.DataFrame(pred) for pred in all_predictions], ignore_index=True)

# Include all columns in all_predictions DataFrame
all_predictions_df = pd.concat([data.drop(columns=["sellingprice"]), all_predictions_df], axis=1)

# Save all predictions to a single CSV file
all_predictions_df.to_csv("all_predictions.csv", index=False)

print("Predictions saved successfully.")


Processing chunk 1...
Average accuracy for chunk 1: 0.9573176517638193
Processing chunk 2...
Average accuracy for chunk 2: 0.9035045390691654
Processing chunk 3...
Average accuracy for chunk 3: 0.9290951543377435
Processing chunk 4...


KeyboardInterrupt: 